In [2]:
import pandas as pd
import holidays
import seaborn as sns
import requests
import numpy as np
from time import sleep
from shutil import copyfile
import random
from scipy import stats

In [2]:
%matplotlib inline
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [3]:
df_data_orig = pd.read_csv("dataset/trip_data_4.csv")
df_trip_orig = pd.read_csv("dataset/trip_fare_4.csv")
Nall = df_data_orig.shape[0]

/home/arditto_trianggada3/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
Nall

15100468

In [5]:
# Nsample = 2000000
Nsample = Nall

In [6]:
# isample = random.sample(range(Nall),Nsample)
isample = range(Nall)

In [7]:
df_data = df_data_orig.loc[isample,:].reset_index(drop=True)
df_trip = df_trip_orig.loc[isample,:].reset_index(drop=True)

In [8]:
df_data.columns = [x.strip() for x in df_data.columns.values]
df_trip.columns = [x.strip() for x in df_trip.columns.values]

df_merge = df_data\
.merge(df_trip, left_on=['medallion','hack_license','vendor_id','pickup_datetime'], right_on=['medallion','hack_license','vendor_id','pickup_datetime'], how='left')

df = df_merge.copy()

In [9]:
BB = (-74.25, -73.75, 40.60, 41.05)

idx = (df.pickup_longitude>=BB[0]) & \
    (df.pickup_longitude<=BB[1]) & \
    (df.dropoff_longitude>=BB[0]) & \
    (df.dropoff_longitude<=BB[1]) &\
    (df.pickup_latitude>=BB[2]) & \
    (df.pickup_latitude<=BB[3]) & \
    (df.dropoff_latitude>=BB[2]) & \
    (df.dropoff_latitude<=BB[3]) 

df = df.loc[idx,:].reset_index(drop=True)

In [10]:
# For this plot and further analysis, we need a function to calculate the distance in miles between locations in lon,lat coordinates.
# This function is based on https://stackoverflow.com/questions/27928/
# calculate-distance-between-two-latitude-longitude-points-haversine-formula 
# return distance in miles
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a)) # 2*R*asin...

# First calculate two arrays with datapoint density per sq mile
n_lon, n_lat = 100, 90 # number of grid bins per longitude, latitude dimension
density_pickup, density_dropoff = np.zeros((n_lat, n_lon)), np.zeros((n_lat, n_lon)) # prepare arrays

# To calculate the number of datapoints in a grid area, the numpy.digitize() function is used. 
# This function needs an array with the (location) bins for counting the number of datapoints
# per bin.
bins_lon = np.zeros(n_lon+1) # bin
bins_lat = np.zeros(n_lat+1) # bin
delta_lon = (BB[1]-BB[0]) / n_lon # bin longutide width
delta_lat = (BB[3]-BB[2]) / n_lat # bin latitude height
bin_width_miles = distance(BB[2], BB[1], BB[2], BB[0]) / n_lon # bin width in miles
bin_height_miles = distance(BB[3], BB[0], BB[2], BB[0]) / n_lat # bin height in miles
for i in range(n_lon+1):
    bins_lon[i] = BB[0] + i * delta_lon
for j in range(n_lat+1):
    bins_lat[j] = BB[2] + j * delta_lat
    
# Digitize per longitude, latitude dimension
inds_pickup_lon = np.digitize(df.pickup_longitude, bins_lon)
inds_pickup_lat = np.digitize(df.pickup_latitude, bins_lat)
inds_dropoff_lon = np.digitize(df.dropoff_longitude, bins_lon)
inds_dropoff_lat = np.digitize(df.dropoff_latitude, bins_lat)


In [11]:
df['pickup_longitude_bin'] = pd.Series(inds_pickup_lon).map(lambda x: bins_lon[x] if x<len(bins_lon) else None)
df['pickup_latitude_bin'] = pd.Series(inds_pickup_lat).map(lambda x: bins_lat[x] if x<len(bins_lat) else None)
df['dropoff_longitude_bin'] = pd.Series(inds_dropoff_lon).map(lambda x: bins_lon[x] if x<len(bins_lon) else None)
df['dropoff_latitude_bin'] = pd.Series(inds_dropoff_lat).map(lambda x: bins_lat[x] if x<len(bins_lat) else None)

In [12]:
df_ll = pd.read_csv("./dataset/map_address.csv")

In [13]:
dln = np.round(bins_lon[1]-bins_lon[0],3)
dlt = np.round(bins_lat[1]-bins_lat[0],3)
minln = bins_lon[0]
minlt = bins_lat[0]
Nln = len(bins_lon)
Nlt = len(bins_lat)

x = {
    'pickup_longitude':-74.001366,
    'pickup_latitude':40.727348,
    'dropoff_longitude':-73.993782,
    'dropoff_latitude':40.727249
}

In [14]:
df['pickup_addr_idx'] = df.apply(lambda x: 
    (int(np.round((x['pickup_longitude'] - bins_lon[0])/dln)))*Nlt + int(np.round((x['pickup_latitude'] - bins_lat[0])/dlt))
,axis=1)

df['dropoff_addr_idx'] = df.apply(lambda x: 
    (int(np.round((x['dropoff_longitude'] - bins_lon[0])/dln)))*Nlt + int(np.round((x['dropoff_latitude'] - bins_lat[0])/dlt))
,axis=1)

In [15]:
df['pickup_addr_zipcode'] = df['pickup_addr_idx'].map(lambda x: df_ll.loc[x,'addr_zipcode'])
df['pickup_addr_place'] = df['pickup_addr_idx'].map(lambda x: df_ll.loc[x,'addr_place'])
df['pickup_addr_neighborhood'] = df['pickup_addr_idx'].map(lambda x: df_ll.loc[x,'addr_neighborhood'])
df['pickup_addr_sublocal'] = df['pickup_addr_idx'].map(lambda x: df_ll.loc[x,'addr_sublocal'])
df['pickup_addr_local'] = df['pickup_addr_idx'].map(lambda x: df_ll.loc[x,'addr_local'])
df['pickup_addr_administrative_area_level_2'] = df['pickup_addr_idx'].map(lambda x: df_ll.loc[x,'addr_administrative_area_level_2'])

df['dropoff_addr_zipcode'] = df['dropoff_addr_idx'].map(lambda x: df_ll.loc[x,'addr_zipcode'])
df['dropoff_addr_place'] = df['dropoff_addr_idx'].map(lambda x: df_ll.loc[x,'addr_place'])
df['dropoff_addr_neighborhood'] = df['dropoff_addr_idx'].map(lambda x: df_ll.loc[x,'addr_neighborhood'])
df['dropoff_addr_sublocal'] = df['dropoff_addr_idx'].map(lambda x: df_ll.loc[x,'addr_sublocal'])
df['dropoff_addr_local'] = df['dropoff_addr_idx'].map(lambda x: df_ll.loc[x,'addr_local'])
df['dropoff_addr_administrative_area_level_2'] = df['dropoff_addr_idx'].map(lambda x: df_ll.loc[x,'addr_administrative_area_level_2'])

In [16]:
df['pickup_to_dropoff_zipcode']  = df.apply(lambda x: "{}_{}".format(x['pickup_addr_zipcode'] ,x['dropoff_addr_zipcode'] ),axis=1)
df['pickup_to_dropoff_place']    = df.apply(lambda x: "{}_{}".format(x['pickup_addr_place']   ,x['dropoff_addr_place']   ),axis=1)
df['pickup_to_dropoff_neighborhood']    = df.apply(lambda x: "{}_{}".format(x['pickup_addr_neighborhood']   ,x['dropoff_addr_neighborhood']   ),axis=1)
df['pickup_to_dropoff_sublocal'] = df.apply(lambda x: "{}_{}".format(x['pickup_addr_sublocal'],x['dropoff_addr_sublocal']),axis=1)
df['pickup_to_dropoff_local'] = df.apply(lambda x: "{}_{}".format(x['pickup_addr_local'],x['dropoff_addr_local']),axis=1)
df['pickup_to_dropoff_administrative_area_level_2'] = df.apply(lambda x: "{}_{}".format(x['pickup_addr_administrative_area_level_2'],x['dropoff_addr_administrative_area_level_2']),axis=1)


In [17]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

df['pickup_month'] = df['pickup_datetime'].dt.month
df['pickup_dayofweek'] = df['pickup_datetime'].dt.weekday_name
df['pickup_hour'] = df['pickup_datetime'].dt.hour
df['dropoff_month'] = df['dropoff_datetime'].dt.month
df['dropoff_dayofweek'] = df['dropoff_datetime'].dt.weekday_name
df['dropoff_hour'] = df['dropoff_datetime'].dt.hour

In [18]:
us_holidays = holidays.UnitedStates()
df['is_public_holiday'] = df['pickup_datetime'].map(lambda x: 1 if x in us_holidays else 0)
df['is_weekend'] = df['pickup_dayofweek'].map(lambda x: 1 if x in ['Saturday','Sunday'] else 0)
df['is_holiday'] = df.apply(lambda x: max(x['is_public_holiday'],x['is_weekend']), axis=1) 

In [19]:
def late_night (row):
    if (row['pickup_hour'] <= 6) or (row['pickup_hour'] >= 20):
        return 1
    else:
        return 0


def night (row):
    if ((row['pickup_hour'] <= 20) and (row['pickup_hour'] >= 16)) and (not(row['pickup_dayofweek'] in ['Saturday','Sunday'])):
        return 1
    else:
        return 0
    
df['trip_night'] = df.apply (lambda x: night(x), axis=1)
df['trip_late_night'] = df.apply (lambda x: late_night(x), axis=1)

In [20]:
# df['business_hour_morning'] = df['pickup_hour'].map(lambda x: 1 if x < 10 else 0)
# df['business_hour_evening'] = df['pickup_hour'].map(lambda x: 1 if x > 18 else 0)

In [21]:
def manhattan(pickup_lat, pickup_long, dropoff_lat, dropoff_long):
    return np.abs(dropoff_lat - pickup_lat) + np.abs(dropoff_long - pickup_long)

df['trip_dist_euclidean'] = df.apply(lambda x: (np.abs(x['pickup_latitude']-x['dropoff_latitude']) ** 2 + np.abs(x['pickup_longitude']-x['dropoff_longitude'])  ** 2) ** 0.5, axis=1)
df['trip_dist_manhattan'] = df.apply(lambda x: manhattan(x['pickup_latitude'], x['pickup_longitude'], x['dropoff_latitude'], x['dropoff_longitude']), axis=1)

In [22]:
df = df.drop(['pickup_addr_idx','dropoff_addr_idx'],axis=1)

In [23]:
df.shape

(14802407, 56)

In [24]:
df.tail(100)

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,pickup_longitude_bin,pickup_latitude_bin,dropoff_longitude_bin,dropoff_latitude_bin,pickup_addr_zipcode,pickup_addr_place,pickup_addr_neighborhood,pickup_addr_sublocal,pickup_addr_local,pickup_addr_administrative_area_level_2,dropoff_addr_zipcode,dropoff_addr_place,dropoff_addr_neighborhood,dropoff_addr_sublocal,dropoff_addr_local,dropoff_addr_administrative_area_level_2,pickup_to_dropoff_zipcode,pickup_to_dropoff_place,pickup_to_dropoff_neighborhood,pickup_to_dropoff_sublocal,pickup_to_dropoff_local,pickup_to_dropoff_administrative_area_level_2,pickup_month,pickup_dayofweek,pickup_hour,dropoff_month,dropoff_dayofweek,dropoff_hour,is_public_holiday,is_weekend,is_holiday,trip_night,trip_late_night,trip_dist_euclidean,trip_dist_manhattan
14802307,2BE8AF8C2AAA9791C731972187AAAE80,5D38E5C34084B1033016C742CEADC68F,VTS,1,NaN,2013-04-16 22:41:00,2013-04-16 22:50:00,1,540,2.03,-73.975510,40.755089,-73.983154,40.774475,CRD,8.5,0.5,0.5,1.20,0.00,10.70,-73.975,40.760,-73.980,40.775,10029.0,Park Ave,NaN,Manhattan,New York,New York County,10023.0,Amsterdam Ave,NaN,Manhattan,New York,New York County,10029.0_10023.0,Park Ave_Amsterdam Ave,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Tuesday,22,4,Tuesday,22,0,0,0,0,1,0.020839,0.027030
14802308,221FE811F798AEF800C704E719E967EC,5F1144A8412B0AFDD60CBB274A0B7B44,VTS,1,NaN,2013-04-16 22:29:00,2013-04-16 22:46:00,2,1020,5.53,-74.003906,40.733452,-73.948997,40.774178,CRD,17.5,0.5,0.5,1.50,0.00,20.00,-74.000,40.735,-73.945,40.775,10014.0,Bleecker St,NaN,Manhattan,New York,New York County,10028.0,E 83rd St,NaN,Manhattan,New York,New York County,10014.0_10028.0,Bleecker St_E 83rd St,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Tuesday,22,4,Tuesday,22,0,0,0,0,1,0.068364,0.095635
14802309,205CCC9B84F0BCA91B461CC53E27BB71,174F00FEDED4FC7517931A15F73D7DB3,VTS,1,NaN,2013-04-16 22:32:00,2013-04-16 22:44:00,3,720,1.55,-73.970657,40.756241,-73.992157,40.759209,CRD,9.0,0.5,0.5,1.90,0.00,11.90,-73.970,40.760,-73.990,40.760,10017.0,E 49th St,NaN,Manhattan,New York,New York County,10036.0,W 45th St,NaN,Manhattan,New York,New York County,10017.0_10036.0,E 49th St_W 45th St,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Tuesday,22,4,Tuesday,22,0,0,0,0,1,0.021704,0.024468
14802310,73A0C178888FDAB55F59FE1A1E640373,8A73F06E0914A0DC1434FA550716CBCD,VTS,1,NaN,2013-04-16 22:22:00,2013-04-16 22:48:00,5,1560,18.33,-73.776833,40.645844,-73.973221,40.690170,CRD,49.0,0.5,0.5,5.00,0.00,55.00,-73.775,40.650,-73.970,40.695,11430.0,Central Terminal Area,Jamaica,Queens,NaN,Queens County,11217.0,NaN,NaN,Brooklyn,NaN,NaN,11430.0_11217.0,Central Terminal Area_nan,Jamaica_Jamaica,Queens_Brooklyn,nan_nan,Queens County_nan,4,Tuesday,22,4,Tuesday,22,0,0,0,0,1,0.201328,0.240714
14802311,D8BC49114D54A67BFC6C8C266400A23E,E806EFD98AE767B02FED18AC658A3061,VTS,1,NaN,2013-04-16 22:32:00,2013-04-16 22:44:00,2,720,2.75,-73.992859,40.746948,-73.975601,40.776958,CSH,11.5,0.5,0.5,0.00,0.00,12.50,-73.990,40.750,-73.975,40.780,10001.0,7 Av/W 25 St,NaN,Manhattan,New York,New York County,10023.0,Terrace Dr,NaN,Manhattan,New York,New York County,10001.0_10023.0,7 Av/W 25 St_Terrace Dr,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Tuesday,22,4,Tuesday,22,0,0,0,0,1,0.034618,0.047268
14802312,70F5983D94E1BA0B217E9F8F447D4382,0099F69B29CD2AC8C547A48E57754C0B,VTS,1,NaN,2013-04-16 22:32:00,2013-04-16 22:45:00,6,780,2.73,-73.966103,40.762905,-73.995537,40.740959,CRD,11.5,0.5,0.5,2.40,0.00,14.90,-73.965,40.765,-73.995,40.745,10065.0,E 64th St,NaN,Manhattan,New York,New York County,10011.0,6th Ave,NaN,Manhattan,New York,New York County,10065.0

In [25]:
# df.to_csv("./dataset/trip_merge_4_fe_sample2000000.csv",index=False)
df.to_csv("./dataset/trip_merge_4_fe_all.csv",index=False)

In [26]:
df.shape[0]/Nsample

0.9802614726907802

In [27]:
df.shape[0]/Nall

0.9802614726907802

In [3]:
df = pd.read_csv("./dataset/trip_merge_4_fe_all.csv",nrows=1000)

In [4]:
len(df.columns.values)

56

In [5]:
df.columns.values

array(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge',
       'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount',
       'pickup_longitude_bin', 'pickup_latitude_bin',
       'dropoff_longitude_bin', 'dropoff_latitude_bin',
       'pickup_addr_zipcode', 'pickup_addr_place',
       'pickup_addr_neighborhood', 'pickup_addr_sublocal',
       'pickup_addr_local', 'pickup_addr_administrative_area_level_2',
       'dropoff_addr_zipcode', 'dropoff_addr_place',
       'dropoff_addr_neighborhood', 'dropoff_addr_sublocal',
       'dropoff_addr_local', 'dropoff_addr_administrative_area_level_2',
       'pickup_to_dropoff_zipcode', 'pickup_to_dropoff_place',
       'pickup_to_dropoff_neighborhood', 'pickup_to_dropoff_sub

In [ ]:
# df.tail(5)